In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import random

import synchrony.PlottingTools as plottingTools
from synchrony.ParameterSet import ParameterSet
import synchrony.DataStorage as dataStorage
import synchrony.DataAnalysis as dataAnalysis
import synchrony.MakeDataframe as makeDataframe
from synchrony import mainClass

In [2]:
file_path_input_params_json = '../input_params.json'
input_param_dict = mainClass.extract_variables_from_input_params_json(file_path_input_params_json)
root_path = input_param_dict["DATA_FOLDER_PATH"]
simulation_location = 'Paper/Fig_03/Fig_3b' #'Paper/X_Appendix/A4_higher_firing_rate/A4c' #'Paper/X_Appendix/A4_higher_firing_rate/A4c' or 'Paper/Fig_4/Fig_4bc'
file_path = os.path.join(root_path, simulation_location)
print('file_path', file_path)
parameter_path = os.path.join(file_path, 'parameter_set.csv')
print('parameter_path', parameter_path)

file_path /home/berger/Data/Synchrony/Paper/Fig_03/Fig_3b
parameter_path /home/berger/Data/Synchrony/Paper/Fig_03/Fig_3b/parameter_set.csv


In [3]:
pinkish_red = (247 / 255, 109 / 255, 109 / 255)
green = (0 / 255, 133 / 255, 86 / 255)
dark_blue = (36 / 255, 49 / 255, 94 / 255)
light_blue = (168 / 255, 209 / 255, 231 / 255)
blue = (55 / 255, 71 / 255, 133 / 255)
yellow = (247 / 255, 233 / 255, 160 / 255)

# Make dataframe

In [4]:
data_frame = makeDataframe.make_dataframe(file_path)

['/home/berger/Data/Synchrony/Paper/Fig_03/Fig_3b/mean_time_interval_CV.pdf',
 '/home/berger/Data/Synchrony/Paper/Fig_03/Fig_3b/CV_with_cell_cycle_5000.pdf',
 '/home/berger/Data/Synchrony/Paper/Fig_03/Fig_3b/CV.pdf',
 '/home/berger/Data/Synchrony/Paper/Fig_03/Fig_3b/8435_vary_hill_Elf_no_time_traces',
 '/home/berger/Data/Synchrony/Paper/Fig_03/Fig_3b/v_init_mean.pdf',
 '/home/berger/Data/Synchrony/Paper/Fig_03/Fig_3b/CV_with_cell_cycle_short.pdf',
 '/home/berger/Data/Synchrony/Paper/Fig_03/Fig_3b/mean_time_interval.pdf']


# Parameters

In [13]:
t_max = 15
N_max = 2000

# if simulation_location == 'Paper/X_Appendix/A4_higher_firing_rate/A4c':
#     Vb= data_frame["v_birth_th"][0] / data_frame["n_ori_birth"][0] * 0.53
# elif simulation_location == 'Paper/Fig_4/Fig_4bc':
    
Vb= data_frame["v_birth_th"][0] / data_frame["n_ori_birth"][0]
# else:
#     print("problem with file directory")

## Simulations to obtain average time intervall

In [14]:
def calcuate_max_rate_given_n_eff(n, growth_rate, v_init, v_b):
    return n*growth_rate * np.log(2)/np.log((2 * v_init**n)/(v_b**n+v_init**n))

def calculate_approx_opening_prob_of_time(t, rate_growth, v_birth, hill_coeff_eff, crit_vol):
    vol= v_birth * np.exp(rate_growth*t)
    return vol**hill_coeff_eff/(vol**hill_coeff_eff+ crit_vol**hill_coeff_eff)

def calculate_whether_initiate(rate, time_step):
    prob = rate * time_step
    random_number = random.random()
    return prob >= random_number

def make_list_intervals_new(t_max, time_step, rate_growth, v_birth, hill_coeff_eff, crit_vol, firing_rate_0):
    n_simu = 2000
    list_t_init_tupel = []
    time = np.arange(0, t_max, time_step)
    approx_opening_probability_of_time = calculate_approx_opening_prob_of_time(time, rate_growth, v_birth, hill_coeff_eff, crit_vol)
    for simu_i in range(0, n_simu):
        init_events = []
        for n in range(0,2):
            for n_step in range(1, time.size): # start at one because 0 is the initial condition
                if calculate_whether_initiate(firing_rate_0 * approx_opening_probability_of_time[n_step], time_step):
                    init_events.append(time[n_step])
                    break
        list_t_init_tupel.append(init_events)
    delta_t = [abs(item[1]-item[0]) for item in list_t_init_tupel]
    return delta_t

def make_list_intervals_double_function(t_max, time_step, rate_growth, v_birth, hill_activation_potential, hill_origin_opening, v_init_th, f_crit, firing_rate_0):
    n_simu = 2000
    list_t_init_tupel = []
    time = np.arange(0, t_max, time_step)
    volume = v_birth * np.exp(time * rate_growth)
    activation_potential = 1/(1 + (v_init_th / volume)** hill_activation_potential)
    origin_opening_prob = 1/(1+(f_crit / activation_potential)**hill_origin_opening)
#     approx_opening_probability_of_time = calculate_approx_opening_prob_of_time(time, rate_growth, v_birth, hill_coeff_eff, crit_vol)
    for simu_i in range(0, n_simu):
        init_events = []
        for n in range(0,2):
            for n_step in range(1, time.size): # start at one because 0 is the initial condition
                if calculate_whether_initiate(firing_rate_0 * origin_opening_prob[n_step], time_step):
                    init_events.append(time[n_step])
                    break
        list_t_init_tupel.append(init_events)
    delta_t = [abs(item[1]-item[0]) for item in list_t_init_tupel]
    return delta_t

## Calculate the average time intervall between two initiation events

In [15]:
import scipy.special as sc

def calculate_survival_probability_indefinite(time_, growth_rate_, effective_hill_coeff_, v_init_, V_b_, firing_rate_0_):
    return np.exp(- firing_rate_0_ / (growth_rate_ * effective_hill_coeff_) 
                  * np.log(v_init_**effective_hill_coeff_ + (V_b_ * np.exp(growth_rate_ * time_))**effective_hill_coeff_))

def calculate_survival_probability_definite(time_, t_0_, growth_rate_, effective_hill_coeff_, v_init_, V_b_, firing_rate_0_):
    return np.exp(- firing_rate_0_ / (growth_rate_ * effective_hill_coeff_) 
                  * np.log((v_init_**effective_hill_coeff_ + (V_b_ * np.exp(growth_rate_ * time_))**effective_hill_coeff_)/(v_init_**effective_hill_coeff_ + (V_b_ * np.exp(growth_rate_ * t_0_))**effective_hill_coeff_)))


def w_t_indef(time_, t_0_, growth_rate_, effective_hill_coeff_, v_init_, V_b_, firing_rate_0_):
    vol = V_b_ * np.exp(growth_rate_* time_)
    prefactor = firing_rate_0_ * vol**effective_hill_coeff_ / (vol**effective_hill_coeff_ + v_init_**effective_hill_coeff_ )
    return prefactor * calculate_survival_probability_definite(time_, 
                                                                 t_0_,
                                                                 growth_rate_, 
                                                                 effective_hill_coeff_, 
                                                                 v_init_, 
                                                                 V_b_,
                                                                 firing_rate_0_)

def double_integrate_two_events(growth_rate_, v_init_, V_b_, n_, rate, t_d_):
    f = lambda t2, t1, growth_rate, v_init, V_b, n, k_0: w_t_indef(t1, 0, growth_rate, n, v_init, V_b, rate) * w_t_indef(t2, 0,  growth_rate, n, v_init, V_b, rate) * abs(t1-t2)
    #factor(growth_rate, v_init, V_b, n, 2 * k_0, t1) * factor(growth_rate, v_init, V_b, n, k_0, t2) * (t2-t1)
    return integrate.dblquad(f, 0, t_d_, 0, t_d_, args=(growth_rate_, 
                                                         v_init_, 
                                                         V_b_, 
                                                         n_, 
                                                         rate,))[0]

def double_integrate_two_events_v2(growth_rate_, v_init_, V_b_, n_, rate, t_d_):
    f = lambda t2, t1, growth_rate, v_init, V_b, n, k_0: w_t_indef(t1, 0,  growth_rate, n, v_init, V_b, 2*rate) * w_t_indef(t2, t1, growth_rate, n, v_init, V_b, rate) * (t2-t1)
    #factor(growth_rate, v_init, V_b, n, 2 * k_0, t1) * factor(growth_rate, v_init, V_b, n, k_0, t2) * (t2-t1)
    return integrate.dblquad(f, 0, t_d_, lambda t1: t1, t_d_, args=(growth_rate_, 
                                                         v_init_, 
                                                         V_b_, 
                                                         n_, 
                                                         rate,))[0]
def w_t_simple(time_, rate_):
    return rate_ * np.exp(- rate_ * time_)

def double_integrate_two_events_simple(rate, t_d_):
    f = lambda t2, t1, rate: w_t_simple(t1, rate) * w_t_simple(t2, rate) * abs(t2-t1)
    #factor(growth_rate, v_init, V_b, n, 2 * k_0, t1) * factor(growth_rate, v_init, V_b, n, k_0, t2) * (t2-t1)
    return integrate.dblquad(f, 0, t_d_, 0, t_d_, args=(rate,))[0]

def double_integrate_two_events_simple_2(rate, t_d_):
    f = lambda t2, t1, rate: w_t_simple(t1, 2* rate) * w_t_simple(t2, rate) * (t2-t1)
    #factor(growth_rate, v_init, V_b, n, 2 * k_0, t1) * factor(growth_rate, v_init, V_b, n, k_0, t2) * (t2-t1)
    return integrate.dblquad(f, 0, t_d_,  lambda t1: t1, t_d_, args=(rate,))[0]

def integrate_mean_time(growth_rate_, v_init_, V_b_, n_, k_0_, t_d_):
    f = lambda t1, growth_rate, v_init, V_b, n, k_0: w_t_indef(t1, 0, growth_rate, n, v_init, V_b, k_0) * t1
    return integrate.quad(f, 0, t_d_, args=(growth_rate_, 
                                                         v_init_, 
                                                         V_b_, 
                                                         n_, 
                                                         k_0_,))[0]

def integrate_inner_conditional_time(growth_rate_, v_init_, V_b_, n_, k_0_, t_d_, t1):
    f = lambda t2, growth_rate, v_init, V_b, n, k_0, t1: w_t_indef(t2, 0,  growth_rate, n, v_init, V_b, k_0) * (t2-t1)
    return integrate.quad(f, t1, t_d_, args=(growth_rate_, 
                                                         v_init_, 
                                                         V_b_, 
                                                         n_, 
                                                         k_0_,
                                                     t1,))[0]

def average_asynch_time_cell_cycle_simu(row):
    dataframe_time_traces = pd.read_hdf(row.path_dataset, key='dataset_time_traces')
    dataframe_division_events = pd.read_hdf(row.path_dataset, key='dataset_div_events')
    V_d = dataframe_division_events['v_d']
    n_oris = dataframe_time_traces['n_ori']
    n_oris_synchr = n_oris[(n_oris ==2.0) | (n_oris ==4.0) | (n_oris ==8.0) | (n_oris ==16.0)]
    n_oris_2 = n_oris[(n_oris ==2.0)]
    n_oris_4 = n_oris[(n_oris ==4.0)]
    fraction_asynch = (n_oris.size - n_oris_synchr.size) / n_oris.size
    print('fraction asynch', fraction_asynch)
    print('fraction 2', n_oris_2.size/ n_oris.size)
    print('fraction 4', n_oris_4.size/ n_oris.size)
    time_asynchronous = fraction_asynch / row.doubling_rate * 60
    return time_asynchronous

def calculate_CV_theoretical(growth_rate_, v_init_, V_b_, n_, rate, t_d_):
    
    f = lambda t1, growth_rate, v_init, V_b, n, k_0: w_t_indef(t1, 0, growth_rate, n, v_init, V_b, rate) * (V_b * np.exp(growth_rate * t1))
    f2 = lambda t1, growth_rate, v_init, V_b, n, k_0: w_t_indef(t1, 0, growth_rate, n, v_init, V_b, rate) * (V_b * np.exp(growth_rate * t1))**2
    mean = integrate.quad(f, 0, t_d_, args=(growth_rate_, 
                                                         v_init_, 
                                                         V_b_, 
                                                         n_, 
                                                         rate,))[0]
    second_moment = integrate.quad(f2, 0, t_d_, args=(growth_rate_, 
                                                         v_init_, 
                                                         V_b_, 
                                                         n_, 
                                                         rate,))[0]
    print(np.sqrt(second_moment - mean**2), mean)
    return np.sqrt(second_moment - mean**2)/mean


In [16]:
# data_frame.loc[:, 'n_eff'] = data_frame.apply(lambda row:row.hill_activation_potential * row.hill_origin_opening /2
#                                   , axis = 1)

In [17]:
# data_frame['n_eff']

In [18]:
data_frame.loc[:, 'mean_time_intervals_min'] = data_frame.apply(lambda row: np.mean(make_list_intervals_new(t_max, 
                                  row.time_step, 
                                  row.rate_growth,
                                  Vb,
                                  row.n_eff,
                                  row.v_init_th,
                                  row.origin_open_and_firing_rate
                                 ))*60, axis = 1)

/home/berger/.virtualenvs/synchrony_project/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in power
  
/home/berger/.virtualenvs/synchrony_project/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


In [19]:
data_frame.loc[:, 'theoretical_mean_interval'] = data_frame.apply(lambda row: double_integrate_two_events(row.rate_growth,
                                  row.v_init_th,
                                  Vb,
                                  row.n_eff,
                                  row.origin_open_and_firing_rate,
                                  t_max
                                 )*60, axis = 1)

/home/berger/.virtualenvs/synchrony_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in double_scalars
  
/home/berger/.virtualenvs/synchrony_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
/home/berger/.virtualenvs/synchrony_project/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in double_scalars
  if __name__ == '__main__':
/home/berger/.virtualenvs/synchrony_project/lib/python3.7/site-packages/scipy/integrate/quadpack.py:880: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subra

In [20]:
data_frame.loc[:, 'mean_time_intervals_cell_cycle'] = data_frame.apply(lambda row: average_asynch_time_cell_cycle_simu(row), axis = 1)    

KeyError: 'No object named dataset_time_traces in the file'

# Plot average time and CV

## Average initiation time

In [ ]:
from matplotlib import cm
sns.set(style="ticks")
sns.set_context("poster")
palette='BuPu'

ylorbr = cm.get_cmap('BuPu', 100)

x = ylorbr(np.linspace(0,1,10))
fig, ax = plt.subplots(figsize=(7,7))
ax.set(xlabel=r'$n_{\rm eff}$', ylabel=r'$\langle \Delta t \rangle$')
# ax.plot(n_eff, mean_time_intervals_min, color='r', label='simple simulation')
# ax.plot(n_eff, np.array(theoretical_mean_interval), color='b', label='theory')
# ax.plot(n_eff, np.array(theoretical_mean_interval_v2), color='g', label='theory 2')

# sorted_data_frame = 

# plt.plot(time,approx_open_prob, linestyle = 'dotted', color= color_list[counter])
# plt.plot(time,approx_open_prob, linestyle = 'dotted', color= color_list[counter])
sns.lineplot(
    x='n_eff', 
    y='mean_time_intervals_min',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     dashes=[(4, 1), ''],
#     markers=True,
    color=x[3],
#     style='theoretical_prediction',
    linewidth = 3.5,
    ax=ax
);

sns.lineplot(
    x='n_eff', 
    y='theoretical_mean_interval',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     linestyle="dashed",
#     markers=True,
    color=x[7],
#     style='theoretical_prediction',
    linewidth = 3.5,
    legend = False,
    ax=ax
);

# sns.lineplot(
#     x='n_eff', 
#     y='mean_time_intervals_cell_cycle',
#     data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
# #     hue='Hill coefficients:',
# #     hue='deactivation rate:',
#     linestyle="-.",
#     markers=True,
#     palette= palette,
# #     style='theoretical_prediction',
#     linewidth = 3.5,
#     legend = False,
#     ax=ax
# );

ax.margins(0)

ax.axhline(y=4, color='grey', linestyle='dotted')
ax.axhline(y=3, color='grey', linestyle='dotted')
ax.axvline(x=29, color='grey', linestyle='dotted')
ax.axvline(x=38, color='grey', linestyle='dotted')
ax.set(ylim=(0, 13))
# plt.legend()
plt.savefig(file_path + '/mean_time_interval.pdf', format='pdf', bbox_inches='tight')

# Compare CV

## CV from simple simulations

In [ ]:
def simulations_CV(t_max, time_step, rate_growth, v_birth, firing_rate_0, hill_coeff_eff, crit_vol):
    n_simu = 5000
    list_v_init = []
    time = np.arange(0, t_max, time_step)
    volume = v_birth * np.exp(time * rate_growth)
    probability = calculate_approx_opening_prob_of_time(time, rate_growth, v_birth, hill_coeff_eff, crit_vol)
    for simu_i in range(0, n_simu):
        for n_step in range(1, time.size): # start at one because 0 is the initial condition
            if calculate_whether_initiate(firing_rate_0 * probability[n_step], time_step):
                list_v_init.append(volume[n_step]*2)
                break
    return np.std(np.array(list_v_init))/ np.mean(np.array(list_v_init))

In [ ]:
data_frame.loc[:, 'CV_simulation'] = data_frame.apply(lambda row: simulations_CV(t_max, 
                                  row.time_step, 
                                  row.rate_growth,
                                  Vb,
                                  row.origin_firing_rate,
                                  row.n_eff,
                                  row.v_init_th
                                 ), axis = 1)

## CV from cell cycle simulations

In [ ]:
def calculate_CV(row):
    data_frame_init = pd.read_hdf(row.path_dataset, key='dataset_init_events')
    initiation_volumes = np.array(data_frame_init['v_init'])
    return np.std(initiation_volumes)/np.mean(initiation_volumes)

data_frame.loc[:, 'CV'] = data_frame.apply(lambda row: calculate_CV(row), axis = 1)

## Theoretical CV

In [ ]:
data_frame.loc[:, 'theoretical_CV'] = data_frame.apply(lambda row: calculate_CV_theoretical(row.rate_growth,
                                  row.v_init_th,
                                  Vb,
                                  row.n_eff,
                                  row.origin_open_and_firing_rate,
                                  t_max
                                 ), axis = 1)

## Plot CV

In [ ]:
from matplotlib import cm
sns.set(style="ticks")
sns.set_context("poster")
palette='BuPu'

ylorbr = cm.get_cmap('BuPu', 100)

x = ylorbr(np.linspace(0,1,10))
fig, ax = plt.subplots(figsize=(7,7))
ax.set(xlabel=r'$n_{\rm eff}$', ylabel=r'CV')

sns.lineplot(
    x='n_eff', 
    y='CV',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     linestyle="dashed",
#     markers=True,
    color=x[4],
#     style='theoretical_prediction',
    linewidth = 3.5,
    legend = False,
    ax=ax
);

sns.lineplot(
    x='n_eff', 
    y='CV_simulation',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     linestyle="dashed",
#     markers=True,
    color=x[7],
#     style='theoretical_prediction',
    linewidth = 3.5,
    legend = False,
    ax=ax
);

sns.lineplot(
    x='n_eff', 
    y='theoretical_CV',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     linestyle="dashed",
#     markers=True,
    color=x[2],
#     style='theoretical_prediction',
    linewidth = 3.5,
    legend = False,
    ax=ax
);


ax.margins(0)

ax.axhline(y=0.048, color='grey', linestyle='dotted')
ax.axhline(y=0.062, color='grey', linestyle='dotted')
ax.axvline(x=29, color='grey', linestyle='dotted')
ax.axvline(x=38, color='grey', linestyle='dotted')
ax.set(ylim=(0, 0.2))
# plt.legend()
plt.savefig(file_path + '/CV.pdf', format='pdf', bbox_inches='tight')

## Plot with two axes

In [ ]:
from matplotlib import cm
sns.set(style="ticks")
sns.set_context("poster")
palette='BuPu'

ylorbr = cm.get_cmap('BuPu', 100)

x = ylorbr(np.linspace(0,1,10))
fig, ax = plt.subplots(figsize=(7,7))
ax.set(xlabel=r'$n_{\rm eff}$', ylabel=r'$\langle \Delta t \rangle$ [min]')

sns.lineplot(
    x='n_eff', 
    y='theoretical_mean_interval',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     linestyle="dashed",
#     markers=True,
    color=x[7],
#     style='theoretical_prediction',
    linewidth = 3.5,
    legend = False,
    ax=ax
);

ax.yaxis.label.set_color(x[7])
ax.tick_params(axis='y', colors=x[7])

ax2 = plt.twinx()
sns.lineplot(
    x='n_eff', 
    y='theoretical_CV',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     linestyle="dashed",
#     markers=True,
    color=x[4],
#     style='theoretical_prediction',
    linewidth = 3.5,
    legend = False,
    ax=ax2
);

ax2.set(ylabel=r'CV')
ax2.yaxis.label.set_color(x[4])
ax2.tick_params(axis='y', colors=x[4])
# ax2.set(ylim=(0, 0.2))
ax2.set(ylim=(0, 0.16))

ax.margins(0)

ax.axhline(y=4, color=x[7], linestyle='dotted', alpha=0.5)
ax.axhline(y=3, color=x[7], linestyle='dotted', alpha=0.5)
ax.set_yticks([0, 2, 3, 4, 6, 8, 10, 12])
ax.set_yticklabels([r'0',r'2', r'$\langle \Delta t_{\rm exp} \rangle$', r'$\Delta t_{\rm exp}^{\rm max}$', '6', r'8', '10', '12'])
# ax.fill_between(np.arange(np.amin(np.array(data_frame['n_eff'])),np.amin(np.array(data_frame['n_eff'])), np.array(data_frame['n_eff']).size) , 3, 4, color=x[7], linestyle='dotted', alpha=0.5)

# ax2.axhline(y=0.05, color=x[4], linestyle='dotted', alpha=0.5)
# ax2.axhline(y=0.07, color=x[4], linestyle='dotted', alpha=0.5)

# ax.axvline(x=29, color='grey', linestyle='dotted')
# ax.axvline(x=38, color='grey', linestyle='dotted')
# ax.set(ylim=(0, 12))
ax.axvline(x=20, color='grey', linestyle='dotted')
ax.axvline(x=27, color='grey', linestyle='dotted')
ax.set(ylim=(0, 10))
# plt.legend()
plt.savefig(file_path + '/mean_time_interval_CV.pdf', format='pdf', bbox_inches='tight')

## Plot fraction within 50 percent volume change (to compare to Elf)

In [ ]:

def calculate_fraction_within_50_pc_volume_change(v_division_average, v_birth_average, initiation_volumes):
    print(v_birth_average)
    print(np.mean(initiation_volumes))
    delta_50 = 0.5 * (v_division_average-v_birth_average)    
    vol_50_percent = initiation_volumes[(initiation_volumes > np.mean(initiation_volumes)- 0.5 * delta_50 ) & (initiation_volumes < np.mean(initiation_volumes)+ 0.5 * delta_50)]
    return vol_50_percent.size/initiation_volumes.size

def calculate_fraction_within_50_pc_volume_change_data_frame(row):
    data_frame_init = pd.read_hdf(row.path_dataset, key='dataset_init_events')
    initiation_volumes = np.array(data_frame_init['v_init'])
    data_frame_division = pd.read_hdf(row.path_dataset, key='dataset_div_events')
    v_birth_average = np.mean(np.array(data_frame_division['v_b']))
    v_division_average = np.mean(np.array(data_frame_division['v_d']))
    return calculate_fraction_within_50_pc_volume_change(v_division_average, v_birth_average, initiation_volumes)

In [ ]:
data_frame.loc[:, 'fract_within_50_vol_change_cell_cycle'] = data_frame.apply(lambda row: calculate_fraction_within_50_pc_volume_change_data_frame(row), axis = 1)


In [ ]:
def simulations_f_50(t_max, time_step, rate_growth, v_birth, firing_rate_0, hill_coeff_eff, crit_vol):
    n_simu = 5000
    list_v_init = []
    time = np.arange(0, t_max, time_step)
    volume = v_birth * np.exp(time * rate_growth)
    probability = calculate_approx_opening_prob_of_time(time, rate_growth, v_birth, hill_coeff_eff, crit_vol)
    for simu_i in range(0, n_simu):
        for n_step in range(1, time.size): # start at one because 0 is the initial condition
            if calculate_whether_initiate(firing_rate_0 * probability[n_step], time_step):
                list_v_init.append(volume[n_step])
                break
    return calculate_fraction_within_50_pc_volume_change(2*v_birth, v_birth, np.array(list_v_init))

In [ ]:
data_frame.loc[:, 'fract_within_50_vol_change'] = data_frame.apply(lambda row: simulations_f_50(t_max, 
                                  row.time_step, 
                                  row.rate_growth,
                                  Vb,
                                  row.origin_firing_rate,
                                  row.n_eff,
                                  row.v_init_th
                                 ), axis = 1)

In [ ]:
from matplotlib import cm
sns.set(style="ticks")
sns.set_context("poster")
palette='BuPu'

ylorbr = cm.get_cmap('BuPu', 100)

x = ylorbr(np.linspace(0,1,10))
fig, ax = plt.subplots(figsize=(7,7))
ax.set(xlabel=r'$n_{\rm eff}$', ylabel=r'$f_{50}$')

sns.lineplot(
    x='n_eff', 
    y='fract_within_50_vol_change_cell_cycle',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     linestyle="dashed",
#     markers=True,
    color=x[7],
#     style='theoretical_prediction',
    linewidth = 3.5,
    legend = False,
    ax=ax
);

sns.lineplot(
    x='n_eff', 
    y='fract_within_50_vol_change',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     linestyle="dashed",
#     markers=True,
    color=x[4],
#     style='theoretical_prediction',
    linewidth = 3.5,
    legend = False,
    ax=ax
);

ax.margins(0)

ax.axhline(y=0.95, color='grey', linestyle='dotted')
# ax.axhline(y=0.027, color='grey', linestyle='dotted')
ax.axvline(x=29, color='grey', linestyle='dotted')
ax.axvline(x=38, color='grey', linestyle='dotted')
# ax.set(ylim=(0, 0.13))
# plt.legend()
plt.savefig(file_path + '/CV.pdf', format='pdf', bbox_inches='tight')

# Compare mean

In [ ]:
def simulations_mean(t_max, time_step, rate_growth, v_birth, firing_rate_0, hill_coeff_eff, crit_vol):
    n_simu = 5000
    list_v_init = []
    time = np.arange(0, t_max, time_step)
    volume = v_birth * np.exp(time * rate_growth)
    probability = calculate_approx_opening_prob_of_time(time, rate_growth, v_birth, hill_coeff_eff, crit_vol)
    for simu_i in range(0, n_simu):
        for n_step in range(1, time.size): # start at one because 0 is the initial condition
            if calculate_whether_initiate(firing_rate_0 * probability[n_step], time_step):
                list_v_init.append(volume[n_step])
                break
    return np.mean(np.array(list_v_init))

In [ ]:
data_frame.loc[:, 'mean_simulation'] = data_frame.apply(lambda row: simulations_mean(t_max, 
                                  row.time_step, 
                                  row.rate_growth,
                                  row.v_birth_th/ row.n_ori_birth,
                                  row.origin_firing_rate,
                                  row.n_eff,
                                  row.v_init_th
                                 ), axis = 1)

In [ ]:
def waiting_time_dist_indef(time_, growth_rate_, effective_hill_coeff_, v_init_, V_b_, firing_rate_0_):
    vol = V_b_ * np.exp(growth_rate_* time_)
    prefactor = firing_rate_0_ * vol**effective_hill_coeff_ / (vol**effective_hill_coeff_ + v_init_**effective_hill_coeff_ )
    return prefactor * calculate_survival_probability_indefinite(time_, 
                                                                 growth_rate_, 
                                                                 effective_hill_coeff_, 
                                                                 v_init_, 
                                                                 V_b_,
                                                                 firing_rate_0_)

def integrand_mean(time, growth_rate_, v_init_, V_b_, n_, rate):
    return waiting_time_dist_indef(time, growth_rate_, n_, v_init_, V_b_, rate) * time

def calculate_mean_theoretical(growth_rate_, v_init_, V_b_, n_, rate, t_d_):
    result = integrate.quad(lambda x: integrand_mean(x, growth_rate_, v_init_, V_b_,n_, rate), 0, t_d_)
    print(result)
    return result

In [ ]:
data_frame.loc[:, 'theoretical_mean'] = data_frame.apply(lambda row: calculate_mean_theoretical(row.rate_growth,
                                  row.v_init_th,
                                  Vb,
                                  row.n_eff,
                                  row.origin_open_and_firing_rate,
                                  t_max
                                 )[0], axis = 1)

In [ ]:
from matplotlib import cm
sns.set(style="ticks")
sns.set_context("poster")
palette='BuPu'

ylorbr = cm.get_cmap('BuPu', 100)

x = ylorbr(np.linspace(0,1,10))
fig, ax = plt.subplots(figsize=(7,7))
ax.set(xlabel=r'$n_{\rm eff}$', ylabel=r'CV')

sns.lineplot(
    x='n_eff', 
    y='mean_simulation',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     linestyle="dashed",
#     markers=True,
    color=x[7],
#     style='theoretical_prediction',
    linewidth = 3.5,
    legend = False,
    ax=ax
);
sns.lineplot(
    x='n_eff', 
    y='theoretical_mean',
    data=data_frame.sort_values(by=["hill_activation_potential"], ascending=False),
#     hue='Hill coefficients:',
#     hue='deactivation rate:',
#     linestyle="dashed",
#     markers=True,
    color=x[4],
#     style='theoretical_prediction',
    linewidth = 3.5,
    legend = False,
    ax=ax
);

ax.margins(0)

# ax.axhline(y=0.048, color='grey', linestyle='dotted')
# ax.axhline(y=0.062, color='grey', linestyle='dotted')
# ax.axvline(x=29, color='grey', linestyle='dotted')
# ax.axvline(x=38, color='grey', linestyle='dotted')
# ax.set(ylim=(0, 0.2))
# plt.legend()
plt.savefig(file_path + '/v_init_mean.pdf', format='pdf', bbox_inches='tight')